<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/Scaled_YOLOv4_Pytorch_modificaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 將Colab與google_drive綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 從Github下載Scaled_YOLOv4資料庫
!git clone https://github.com/roboflow-ai/ScaledYOLOv4.git  # clone repo
%cd /content/ScaledYOLOv4/

# 確認yolov4-large branch
!git checkout yolov4-large

Cloning into 'ScaledYOLOv4'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 389.37 KiB | 4.75 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/ScaledYOLOv4
Branch 'yolov4-large' set up to track remote branch 'yolov4-large' from 'origin'.
Switched to a new branch 'yolov4-large'


In [ ]:
# 導入需求套件和設備配置確認
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16160MB, multi_processor_count=80)


In [ ]:
# 安裝並啟動cuda顯卡優化套件
%cd /content/
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install

/content
Cloning into 'mish-cuda'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 10.44 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/mish-cuda
/usr/lib/python3.7/distutils/extension.py:131: UserWarning: Unknown Extension options: 'headers'
  warnings.warn(msg)
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/mish_cuda
copying src/mish_cuda/__init__.py -> build/lib.linux-x86_64-3.7/mish_cuda
running egg_info
creating src/mish_cuda.egg-info
writing src/mish_cuda.egg-info/PKG-INFO
writing dependency_links to src/mish_cuda.egg-info/dependency_links.txt
writing requirements to src/mish_cuda.egg-info/requires.txt
writing top-level names to src/mish_cuda.egg-info/top_level.txt
writing manifest file 'src/mish_c

In [ ]:
# 下載安裝PyYAM(為python的yaml檔案的解析套件)
!pip install -U PyYAML

     |████████████████████████████████| 645kB 14.4MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# 轉換當前路徑目錄
# 下載資料集,並於google_drive取得壓縮檔的id
# id取得方法,到google_drive指定壓縮檔,滑鼠右鍵選擇取得連結,更改權限由限制->知道連結的使用者,複製連結在筆記本上
# https://drive.google.com/file/d/這裡就是壓縮檔的id/view?usp=sharing

%cd /content
!gdown --id '196DvWOY8jQZe9m6D2FFPeUBTpPSXsap0' -O dataset.zip

/content
Downloading...
From: https://drive.google.com/uc?id=196DvWOY8jQZe9m6D2FFPeUBTpPSXsap0
To: /content/dataset.zip
275MB [00:04, 68.2MB/s]


In [ ]:
# 解壓縮,注意壓縮檔名稱需與代碼一致
# 解壓縮完,於右邊選擇檔案,看有無已解壓縮的資料夾

!unzip -q dataset.zip; rm dataset.zip

In [ ]:
# 確定檔案位置和類別數

%cd /content
%cat data.yaml

/content
train: ../train/images
val: ../valid/images

nc: 33
names: ['bacon', 'bagel', 'baguette', 'baozi', 'burger', 'cereal', 'chips', 'cong_you_bing', 'corn_soup', 'croissant', 'danbing', 'drinks', 'fantuan', 'fried_baozi', 'fried_egg', 'guotie', 'ham', 'hash_brown', 'hotdog', 'pancake', 'radish_cake', 'salad', 'sandwich', 'shaobing', 'sliced_bread', 'steamed_bread', 'tea_egg', 'teppanyaki_noodle', 'triangle_rice_ball', 'waffle', 'water_rice', 'xiaolongbao', 'you_tiao']

In [ ]:
# 根據data.yaml來定義訓練的類別數

import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# 讀取ScaledYOLOv4資料夾內yolov4的模型框架

%cat /content/ScaledYOLOv4/models/yolov4-csp.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32

# yolov4-csp backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, Bottleneck, [64]],
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 2, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
   [-1, 8, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
   [-1, 8, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 9-P5/32
   [-1, 4, BottleneckCSP, [1024]],  # 10
  ]

# yolov4-csp head
# na = len(anchors[0])
head:
  [[-1, 1, SPPCSP, [512]], # 11
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [8, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   [-1, 2, Bott

In [ ]:
# 以自己所需求的epoch數量去訓練ScaledYOLOv4, 可自行修改所需要的epochs數量,這裡是1000epochs
# 參數說明: img(輸入的圖片尺寸), batch(batch 大小), epochs(epochs數量), data(.yaml的路徑),cfg(模型的架構),
#       weights(權重,可修改成自己已訓練的權重路徑),name(結果的名稱), nosave(只存最後的checkpoint), cache(內存與圖像高速緩存)

%%time
%cd /content/ScaledYOLOv4/
!python train.py --img 416 --batch 16 --epochs 1000 --data '../data.yaml' --cfg ./models/yolov4-csp.yaml --weights '' --name yolov4-csp-results  --cache

In [ ]:
# 使用tensorboard可視化套件
# 讀取"runs"資料夾下的結果

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# 對於某些較老舊的圖表,如果tensorboard無法顯示,請使用以下套件繪製

from utils.general import plot_results  
plot results.txt as results.png

In [ ]:
# 導入繪圖與顯示套件
from IPython.display import Image, display

# 圖片路徑(如果不對記得修改), 繪圖參數
display(Image('/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/results.png'))  # view results.png

In [ ]:
# 顯示正確Test的照片和其標註的label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/test_batch0_gt.jpg', width=900)

In [ ]:
# 顯示augment後test的照片和其標註的label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/train_batch0.jpg', width=900)

In [ ]:
# 已訓練完的權重會存在runs資料夾, 並檢視檔案與目錄的狀態

%ls runs/

In [ ]:
# 檢視檔案與目錄的狀態

%ls ./runs/exp0_yolov4-csp-results/weights

In [ ]:
# inference存放test資料夾下的照片
# 使用訓練時最好的權重,best.pt(需要注意路徑是否正確)

%cd /content/ScaledYOLOv4/
!python detect.py --weights ./runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt --img 416 --conf 0.4 --source ../test/images

In [ ]:
# 顯示出inference後的圖片(需要注意路徑是否正確)

import glob
from IPython.display import Image, display

for imageName in glob.glob('./inference/output/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#下載權重檔案到google_drive

from google.colab import files
files.download('./runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt')